# Kore — Quickstart on Colab

A pure Rust ML framework with Python bindings. This notebook shows how to:
1. Install Kore from source
2. Create tensors and run autograd
3. Train a simple model
4. Use quantized layers (BitLinear, QuatLinear)

## 1. Install

In [ ]:
# Install Rust toolchain + maturin (only needed once)
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y
!source $HOME/.cargo/env && pip install maturin

In [ ]:
# Clone and build Kore
!git clone https://github.com/KidIkaros/KORE.git
%cd KORE
!source $HOME/.cargo/env && maturin develop --release

## 2. Tensors & Autograd

In [ ]:
import kore
import numpy as np

# Create tensors
x = kore.Tensor.randn([3, 4])
print("shape:", x.shape)
print("dtype:", x.dtype)
print(x.numpy())

In [ ]:
# NumPy interop
arr = np.random.randn(2, 3).astype(np.float32)
t = kore.Tensor(arr)
print("from numpy:", t.shape)
print(t.numpy())

In [ ]:
# Autograd
w = kore.Tensor.randn([4, 2])
w.requires_grad_(True)

# Forward
y = w.matmul(kore.Tensor.ones([2, 1]))
loss = y.sum()
print("loss:", loss.numpy())

# Backward
loss.backward()
print("grad shape:", w.grad.shape)
print("grad:\n", w.grad.numpy())

## 3. Training a Simple Model

In [ ]:
# Simple linear regression: y = 3x + 1
import kore
import numpy as np

# Generate data
np.random.seed(42)
X_np = np.random.randn(64, 1).astype(np.float32)
Y_np = (3.0 * X_np + 1.0 + 0.1 * np.random.randn(64, 1)).astype(np.float32)

X = kore.Tensor(X_np)
Y = kore.Tensor(Y_np)

# Model
model = kore.nn.Linear(1, 1, bias=True)
optimizer = kore.optim.SGD(lr=0.01)

# Training loop
for epoch in range(100):
    pred = model(X)
    loss = kore.functional.mse_loss(pred, Y)
    
    if epoch % 20 == 0:
        print(f"epoch {epoch:3d}  loss={loss.numpy().item():.4f}")

print(f"\nFinal loss: {loss.numpy().item():.4f}")

## 4. Quantized Layers

In [ ]:
# BitLinear: 1.58-bit ternary quantization
bit_layer = kore.nn.BitLinear(256, 128)
print(bit_layer)
print(f"Compression: {bit_layer.compression_ratio():.1f}x")

x = kore.Tensor.randn([4, 256])
out = bit_layer(x)
print(f"Output shape: {out.shape}")

In [ ]:
# QuatLinear: 2-bit quaternary quantization
quat_layer = kore.nn.QuatLinear(256, 128)
print(quat_layer)
print(f"Compression: {quat_layer.compression_ratio():.1f}x")

out = quat_layer(x)
print(f"Output shape: {out.shape}")

## 5. Conv + Pooling

In [ ]:
# Simple CNN forward pass
conv = kore.nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
pool = kore.nn.MaxPool2d(2, stride=2)

img = kore.Tensor.randn([1, 3, 32, 32])
h = conv(img)
print(f"After conv: {h.shape}")
h = pool(h)
print(f"After pool: {h.shape}")

## 6. Save & Load

In [ ]:
# Save model weights
sd = {"w": kore.Tensor.randn([4, 4]), "b": kore.Tensor.zeros([4])}
kore.save_state_dict(sd, "demo_weights.safetensors")

# Load them back
loaded = kore.load_state_dict("demo_weights.safetensors")
print("Loaded keys:", list(loaded.keys()))
print("w shape:", loaded["w"].shape)
print("b shape:", loaded["b"].shape)